<a href="https://colab.research.google.com/github/Abudo-S/MindTheGap/blob/main/MindTheGap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
xo